In [2]:
import numpy as np
import pandas as pd
import random
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from joblib import dump
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import itertools
from sklearn.utils import shuffle
from scipy import signal
%matplotlib inline


from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import SelectFdr, chi2

from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score

from utils.svm import preProcess, evaluate_set
from utils.visualize import showMe
from utils.augment import apply_augment
from config.default import *


%load_ext autoreload
%autoreload 2


C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
C:\Users\Nemes\anaconda3\envs\tf\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
def create_labels(X):
    y = []
    for i, r in enumerate(X):
        l = np.ones(X[r].shape[0])*i
        y = y + l.tolist()
    y = np.array(y)
    return y


In [17]:
root_path = 'C:/resources/EMG/'
post_fix = '_1s_cleaned' #'_1s_new' #
classes = settings['classes']
subjects = os.listdir(root_path)
for subject in subjects:
    print(f'Cross validating, leaving out subject {subject}')
    cross_validate(subject)
    break

Cross validating, leaving out subject S001
54 sessions loaded for training
4 sessions loaded for validation
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ...................................C=10, gamma=0.01; total time= 3.1min
[CV] END ...................................C=10, gamma=0.01; total time= 2.9min
[CV] END ...................................C=10, gamma=0.01; total time= 3.1min
[CV] END ...................................C=10, gamma=0.01; total time= 2.6min
[CV] END ...................................C=10, gamma=0.01; total time= 2.7min


  0%|          | 0/4 [00:00<?, ?it/s]

Global accuracy: 83.5%
         Accuracy
Subject          
S001         83.5


In [16]:
def cross_validate(subject):
    # use session4 for validation
    subject_to_val = [subject]
    train_sessions = []
    val_sessions = []

    for subject in os.listdir(root_path):
        for session in os.listdir(os.path.join(root_path,subject)):
            if subject in subject_to_val:
                val_sessions.append(os.path.join(root_path,subject, session))
            else:
                train_sessions.append(os.path.join(root_path,subject, session))
        


    train_records = {}
    for c in classes:
        class_data = []
        for session in train_sessions:
            data = np.load(os.path.join(session,c+post_fix+'.npy'),allow_pickle=True)
            if data.shape[0] != 0:
                class_data.append(data)
            #else:
            #    print(f"No data available for train for class {c}")
        
        train_records[c] = np.concatenate(class_data)
    print(f"{len(train_sessions)} sessions loaded for training")


    val_records = {}
    for c in classes:
        class_data = []
        for session in val_sessions:
            data = np.load(os.path.join(session,c+post_fix+'.npy'),allow_pickle=True)
            if data.shape[0] != 0:
                class_data.append(data)
        if len(class_data) != 0:
            val_records[c] = np.concatenate(class_data)
        #else:
        #    print(f"No data available for validation for class {c}")

    print(f"{len(val_sessions)} sessions loaded for validation")





    n_channels = train_records["Rest"].shape[1]
    input_length = train_records["Rest"].shape[2]

    train_y = create_labels(train_records)
    train_X = np.concatenate((list(train_records.values())), axis=0)

    train_X, train_y = apply_augment(train_X, train_y)
    train_X = train_X.reshape(train_X.shape[0], n_channels*input_length)

    param_grid = {'C': [10], 'gamma': [0.01]}

    grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
    grid.fit(train_X,train_y)
    model = grid.best_estimator_ 
    evaluate_set(model, val_sessions, classes, post_fix, log = False)